In [6]:
import pandas as pd
import os
import numpy as np 
import altair as alt
import pycountry


## Altair Config

In [19]:
def e4e_basic():
    return {
        'config': {
            'axisX': {
                'labelPadding': 6,
                'tickOpacity': 0,
                'title': None,
                'grid': False,
                'ticks': False,
                'labelSeparation': 10
            },
            'axisY': {
                'title': None,
                'grid': False,
                'tickOpacity': 0,
            },
            'title': {
                'fontSize': 16,
                'subtitleFontSize': 12,
                'subtitleFontStyle': 'italic',
                'anchor': 'start',
                'color': 'black'
            },
            'view': {
                'stroke': 'transparent'
            }
        }
    }

# Register the custom theme under a chosen name
alt.themes.register('e4e_basic', e4e_basic)

# Enable the theme
alt.themes.enable('e4e_basic')

ThemeRegistry.enable('e4e_basic')

# Soverign CDS Spread

In [77]:
df = pd.read_excel("data/Damodaran_stern_spreads.xlsx")

iso3s = {}

manual_iso3s = {
    "Congo, Republic of": "COG",
    "China, People's Republic of": "CHN",
    "Bahamas, The": "BHS",
    "Taiwan Province of China": "TWN",
    "Congo, Dem. Rep. of the": "COD",
    "Gambia, The": "GMB",
    "Türkiye, Republic of": "TUR",
    "South Sudan, Republic of": "SSD",
    "Hong Kong SAR": "HKG",
    "Macao SAR": "MAC",
    "Nigeria": "NGA",
    "Niger": "NER",
    'Congo (Democratic Republic' : "COD",
    'Congo (Republic of)' : "COG",
    'Korea, D.P.R': "PRK",

}


for country in df.Country.unique():
    try:
        if country in manual_iso3s:
            iso3s[country] = manual_iso3s[country.strip()]
        else:
            iso3s[country] = pycountry.countries.search_fuzzy(country)[0].alpha_3
    except:
        print(f"'{country}' not found", end=", ")

df["iso3"] = df["Country"].map(iso3s)

full_df = df.copy()


'Abu Dhabi' not found, 'Andorra (Principality of)' not found, 'Antigua & Barbuda' not found, 'Cape Verde' not found, 'Channel Islands' not found, 'Guernsey (States of)' not found, 'Ivory Coast' not found, 'Jersey (States of)' not found, 'Korea, D.P.R.' not found, 'Laos' not found, 'Macau' not found, 'Netherlands Antilles' not found, 'Palestinian Authority' not found, 'Ras Al Khaimah (Emirate of)' not found, 'Sharjah' not found, 'St. Maarten' not found, 'St. Vincent & the Grenadines' not found, 'Swaziland' not found, 'Trinidad &' Tobago' not found, 'Turks & Caicos Islands' not found, 

### Chart: P21 Sovereign CDS Spread

In [78]:
p21_iso3s = ["BGD", "BRA", "CHN", "COD", "EGY", "ETH", "GBR", "DEU", "IND", "IDN", "IRN", "JPN", "MEX", "NGA", "PAK", "PHL", "RUS", "THA", "TUR", "USA", "VNM"]

df = full_df.copy()
df = df[df.iso3.isin(p21_iso3s)]

df = df.dropna(subset=["Sovereign CDS Spread"]).sort_values("Sovereign CDS Spread", ascending=False)

df["Value"] = df["Sovereign CDS Spread"]
df = df[["Country", "Value", "iso3"]]

base = alt.Chart(df).encode(
    x=alt.X("Value:Q", title="", axis=alt.Axis(format=".0%")),
    y=alt.Y("Country:N", sort="-x", axis=alt.Axis(title=None)),
)    

bars = base.mark_bar(color="rgb(9, 32, 95)")

labels = base.mark_text(
    align="left",
    baseline="middle",
    dx=3
).encode(
    text=alt.Text("Value:Q", format=".0%"),
)

chart = bars + labels

chart = chart.properties(
    title = alt.TitleParams(
            "Sovereign CDS Spread",
    subtitle=["January 2024 estimates", "Source: Damodaran, 2024", ""],
    ),

    width=400,
    height=300
)

chart.display()
chart.save("charts/CDS/sovereign_cds_spread.svg")
chart.save("charts/CDS/sovereign_cds_spread.json")
chart.save("charts/CDS/sovereign_cds_spread.png", scale_factor=2.0)



/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = 

In [82]:
[c for c in p21_iso3s if c not in df.iso3.unique()]

['BGD', 'COD', 'IRN', 'RUS']

#### Russian CDS Price

In [75]:
df = pd.read_csv("data/ru_CDS.csv")
# American dates :((
df["Date"] = pd.to_datetime(df["Date"], format="%m/%d/%Y")
df["Value"] = df["Price"].astype(str).str.replace(",", "").astype(float)

base = alt.Chart(df).encode(
    x=alt.X("Date:T", title="", axis=alt.Axis(format="%b %Y")),
    y=alt.Y("Value:Q", title=""),
)

line = base.mark_line(color="rgb(9, 32, 95)", interpolate="step-after")

# add a February 24th 2022 rule - the day of the invasion
rule = alt.Chart(pd.DataFrame({"Date": [pd.to_datetime("2022-02-24")]})).mark_rule(color="black", size=2, strokeDash=[3,3]).encode(x="Date:T")

invasion_label = alt.Chart(pd.DataFrame({"Date": [pd.to_datetime("2022-02-24")], "Value": [df["Value"].max()]})).mark_text(
    text="Invasion of Ukraine",
    color="black",
    dy=-15,
).encode(
    x="Date:T",
    y="Value:Q",
)

chart = line + rule + invasion_label

chart = chart.properties(
    title = alt.TitleParams(
            "Russia: Credit Default Swap",
    subtitle=["5-Year CDS Price", "Source: investing.com", ""],
    ),

    width=400,
    height=300
)

chart.display()

chart.save("charts/CDS/ru_cds.svg")
chart.save("charts/CDS/ru_cds.json")
chart.save("charts/CDS/ru_cds.png", scale_factor=2.0)

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


### Sovereign Bond Yields

In [76]:
df = pd.read_excel("data/worldgovernmentbonds_yields.xlsx")
df['iso3'] = df['Country'].map(iso3s)
df = df[df.iso3.isin(p21_iso3s)]
df = df[["Country", "Yield▴", "iso3"]]
df = df.rename(columns={"Yield▴": "Value"})
df = df.reset_index(drop=True)
df["Source"] = "https://www.worldgovernmentbonds.com"

manual_yields = [
    {
        "Country": "Ethopia",
        "Value": 0.66,
        "iso3": "ETH",
        "Source" : "https://www.reuters.com/world/africa/ethiopia-edges-towards-default-bondholder-talks-falter-2023-12-08/"
    },
    {
        "Country": "DR Congo",
        "Value": 0.25,
        "iso3": "COD",
        "Source" : "https://cbonds.com/news/2612405/"
    }
]


df = pd.concat([df, pd.DataFrame(manual_yields)])

base = alt.Chart(df).encode(
    x=alt.X("Value:Q", title="", axis=alt.Axis(format=".0%")),
    y=alt.Y("Country:N", sort="-x", axis=alt.Axis(title=None)),
)

bars = base.mark_bar(color="rgb(9, 32, 95)")

labels = base.mark_text(
    align="left",
    baseline="middle",
    dx=3
).encode(
    text=alt.Text("Value:Q", format=".1%"),
)

chart = bars + labels

chart = chart.properties(
    title = alt.TitleParams(
            "Government Bond Yields",
    subtitle=["10-Year Government Bond Yields", "Various Sources", ""],
    ),

    width=300,
    height=400
)

chart.display()
chart.save("charts/bonds/gov_bond_yields.json")
chart.save("charts/bonds/gov_bond_yields.png", scale_factor=2.0)
chart.save("charts/bonds/gov_bond_yields.svg")

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/

In [67]:
df

,Country,Value,iso3,Source
0,Japan,0.00725,JPN,https://www.worldgovernmentbonds.com
1,Vietnam,0.02328,VNM,https://www.worldgovernmentbonds.com
2,Germany,0.02382,DEU,https://www.worldgovernmentbonds.com
3,China,0.02447,CHN,https://www.worldgovernmentbonds.com
4,Thailand,0.02566,THA,https://www.worldgovernmentbonds.com
5,United Kingdom,0.04123,GBR,https://www.worldgovernmentbonds.com
6,United States,0.04190,USA,https://www.worldgovernmentbonds.com
7,Philippines,0.06278,PHL,https://www.worldgovernmentbonds.com
8,Indonesia,0.06719,IDN,https://www.worldgovernmentbonds.com
9,India,0.07114,IND,https://www.worldgovernmentbonds.com
